In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import requests

# Import API key
from api_keys import api_key

In [9]:
oscar_data= pd.read_csv("oscar_data.csv")
oscar_data

,Year,Host,Most Nominations,Most Wins,Best Picture,Nominee 1,Nominee 2,Nominee 3,Nominee 4,Nominee 5,...,Best Costume Design (Black and White),Nominee 1.31,Nominee 2.31,Nominee 3.30,Nominee 4.30,Best Costume Design (Color),Nominee 1.32,Nominee 2.32,Nominee 3.31,Nominee 4.31
0,1927/28,Douglas Fairbanks,Seventh Heaven (5),Seventh Heaven AND Sunrise: A Song of Two Huma...,Wings,The Racket,Seventh Heaven,Unique and Artistic Production- Sunrise: A Son...,Chang: A Drama of the Wilderness,The Crowd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1928/29,William C. DeMille,In Old Arizona AND The Patriot (5),None (No Film Won More Than One Award),The Broadway Melody,Alibi,Hollywood Revue,In Old Arizona,The Patriot,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1929/30,Conrad Nagel,The Love Parade (6),All Quiet on the Western Front AND The Big Hou...,All Quiet on the Western Front,The Big House,Disraeli,The Divorcee,The Love Parade,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1930/31,Lawrence Grant,Cimarron (7),Cimarron (3),Cimarron,East Lynne,The Front Page,Skippy,Trader Horn,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1931/32,Conrad Nagel,Arrowsmith AND The Champ (4),Bad Girl AND The Champ (2),Grand Hotel,Arrowsmith,Bad Girl,The Champ,Five Star Final,One Hour with You,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2019,NaN,Joker (11),Parasite (4),Parasite,1917,Ford v Ferrari,The Irishman,Jojo Rabbit,Joker,...,NaN,NaN,NaN,NaN,NaN,Jacqueline Durran for Little Women,Mayes C. Rubeo for Jojo Rabbit,Mark Bridges for Joker,Sandy Powell and Christopher Peterson for The ...,Arianne Phillips for Once Upon a Time in Holly...
92,2020,NaN,Mank (10),Nomadland (3),Nomadland,The Father,Judas and the Black Messiah,Mank,Minari,Promising Young Woman,...,NaN,NaN,NaN,NaN,NaN,Ann Roth for Ma Rainey's Black Bottom,Alexandra Byrne for Emma.,Trish Summerville for Mank,Bina Daigeler for Mulan,Massimo Cantini Parrini for Pinocchio
93,2021,"Amy Schumer, Regina Hall, and Wanda Sykes",The Power of the Dog (12),Dune (6),CODA,Belfast,Don't Look Up,Drive My Car,Dune,King Richard,...,NaN,NaN,NaN,NaN,NaN,Jenny Beavan for Cruella,Massimo Cantini Parrini for Cyrano,Jacqueline West and Bob Morgan for Dune,Luis Sequeira for Nightmare Alley,Paul Tazewell for West Side Story
94,2022,Jimmy Kimmel,Everything Everywhere All At Once (11),Everything Everywhere All At Once (7),Everything Everywhere All At Once,All Quiet on the Western Front,Avatar: The Way of Water,The Banshees of Inisherin,Elvis,The Fabelmans,...,NaN,NaN,NaN,NaN,NaN,Ruth E. Carter for Black Panther: Wakanda Forever,Mary Zophres for Babylon,Cahterine Martin for Elvis,Shirley Kurata for Everything Everywhere All A...,Jenny Beavan for Mrs. Harris Goes to Paris
